In [3]:
import pandas as pd
import re
from datetime import datetime
import json

## Quick overview

In [5]:
df = pd.read_csv('app/data/raw_messages.csv')

In [6]:
df.columns

Index(['device_id', 'datetime', 'address_ip', 'address_port',
       'original_message_id', 'raw_message'],
      dtype='object')

In [7]:
df.dtypes

device_id              object
datetime                int64
address_ip             object
address_port            int64
original_message_id    object
raw_message            object
dtype: object

In [8]:
df.head()

,device_id,datetime,address_ip,address_port,original_message_id,raw_message
0,0001,1550066999,172.19.0.17,4007,1550070599576-0,"A,51.31830816666667,N,4.315722166666666,E,0.0,..."
1,0001,1550067661,172.19.0.16,4007,1550071261429-0,"A,51.31830816666667,N,4.315722166666666,E,0.0,..."
2,0001,1550067048,172.19.0.17,4007,1550070648974-0,"A,51.3183085,N,4.315720833333334,E,0.0,5.25,15..."
3,0001,1550069034,172.19.0.16,4007,1550072634730-0,"A,51.3183085,N,4.315720833333334,E,0.0,5.25,15..."
4,0001,1550069696,172.19.0.16,4007,1550073296534-0,"A,5$1.31%83085,N&,4.3@15*720833333334@,E,0.0,5..."


In [9]:
df.sample(5)

,device_id,datetime,address_ip,address_port,original_message_id,raw_message
15299,0001,1550067706,172.19.0.16,4007,1550071306082-0,"A,5$1.31%8308666&6666@7,*N,4.31572083@3333334,..."
19801,st-1a2090,1550051935,172.23.0.1,4007,1550055535567-1,"A,5$1.90%25965,N&,5.5@41*472,E,2.98,6@4.79,130..."
22300,st-1a2090,1550069124,172.24.0.1,4007,1550072724767-0,"A,51.81206783333333,N,4.8768365,E,8.77,223.24,..."
15143,st-1a2090,1550056562,172.23.0.1,4007,1550060162902-0,"A,51.90100666666667,N,5.541864,E,0.04,306.32,1..."
1971,st-1a2090,1550079538,172.24.0.1,4007,1550083138847-0,"A,51.69041966666666,N,4.4091,E,0.01,4.81,13021..."


In [10]:
df.isnull().sum()

device_id              0
datetime               0
address_ip             0
address_port           0
original_message_id    0
raw_message            0
dtype: int64

In [11]:
df.nunique()

device_id                  2
datetime               14984
address_ip                12
address_port               1
original_message_id    29047
raw_message            25950
dtype: int64

In [12]:
len(df)

29052

## Feature extraction

In [14]:
def unix_to_human_readable(unix_timestamp, format='%Y-%m-%d %H:%M:%S'):
    try:
        dt = datetime.utcfromtimestamp(unix_timestamp)
        date = dt.strftime(format)
        return date
    except (TypeError, ValueError) as e:
        print(f"{e}")
        return f"Invalid timestamp: {e}"

In [15]:
def clean_message(raw_message):
    # Allow numbers, letters, and specific symbols (.,-)
    clean_message = re.sub(r'[^A-Za-z0-9.,-]', '', raw_message)
    return clean_message

In [16]:
for index, row in df.iterrows():
    timestamp = unix_to_human_readable(row['datetime'])
    df.at[index, 'timestamp'] = timestamp
    raw_message = row['raw_message']
    clean_raw_message = clean_message(raw_message)
    split_raw_message = clean_raw_message.split(',')
    df.at[index, 'status'] = split_raw_message[0]
    df.at[index, 'lat'] = split_raw_message[1]
    df.at[index, 'lat_dir'] = split_raw_message[2]
    df.at[index, 'lon'] = split_raw_message[3]
    df.at[index, 'lon_dir'] = split_raw_message[4]
    df.at[index, 'speed (knots)'] = split_raw_message[5]
    df.at[index, 'course'] = split_raw_message[6]
    df.at[index, 'datestamp'] = split_raw_message[7]
    df.at[index, 'dilution'] = split_raw_message[8]
    df.at[index, 'dilution_dir'] = split_raw_message[9]

C:\Users\adir7\AppData\Local\Temp\ipykernel_9404\537838.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(unix_timestamp)
C:\Users\adir7\AppData\Local\Temp\ipykernel_9404\537838.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(unix_timestamp)


## Cleaning

In [18]:
# Drop rows where status is "V" for invalied messages
df = df[df['status'] != 'V']

In [19]:
# # Assuming df is your DataFrame
# df['speed (knots)'] = pd.to_numeric(df['speed (knots)'], errors='coerce')
# df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
# df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

In [20]:
# Convert columns to numeric, coercing errors to NaN
df['speed (knots)'] = pd.to_numeric(df['speed (knots)'], errors='coerce')
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

# Drop rows where any of these columns have NaN (invalid values)
df = df.dropna(subset=['speed (knots)', 'lat', 'lon'])

## Validating

In [22]:
statuses = set(df['status'])
print(statuses)

{'A'}


In [23]:
# Find rows where both device_id and original_message_id are duplicated (not allowed)
duplicates = df[df.duplicated(subset=['device_id', 'original_message_id'], keep=False)]

print(duplicates)

Empty DataFrame
Columns: [device_id, datetime, address_ip, address_port, original_message_id, raw_message, timestamp, status, lat, lat_dir, lon, lon_dir, speed (knots), course, datestamp, dilution, dilution_dir]
Index: []


In [24]:
len(df)

29016

In [25]:
df.to_csv('cleaned_messages_data.csv', index=False)